# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [71]:
weather_data = pd.read_csv("../WeatherPy/Output_Data/cities.csv")
weather_data

City  Latitude  Longitude  Max Temp  Humidity  Cloudiness  \
0          albany     42.60     -73.97     57.99        82         100   
1        sisimiut     66.94     -53.67     32.97        91         100   
2       jamestown     42.10     -79.24     48.00        93          90   
3      butaritari      3.07     172.79     82.09        76          91   
4    saint george     37.10    -113.58     84.20         9           1   
..            ...       ...        ...       ...       ...         ...   
545        banjar     -8.19     114.97     86.00        84          75   
546      kizukuri     40.81     140.39     64.00        50          20   
547        calama    -22.47     -68.93     64.40        10           0   
548      bilibino     68.05     166.44     22.17        94         100   
549          tual     -5.67     132.75     77.83        88         100   

     Wind Speed Country        Date  
0          3.62      US  1603240803  
1         25.28      GL  1603240946  
2          8.05      US  1603240947  
3         11.68      KI  1603240948  
4          5.82      US  1603240778  
..          ...     ...         ...  
545        2.24      ID  1603241652  
546        2.24      JP  1603241653  
547       16.11      CL  1603241654  
548        2.06      RU  1603241656  
549       12.57      ID  1603241657  

[550 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [72]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humitidy as the weight
locations = weather_data[["Longitude", "Latitude"]]

# Store Humidity
humidity = weather_data["Humidity"]

In [62]:
# Use the Lat and Lng as locations and Humitidy as the weight
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
# Narrow down that DataFrame to find the ideal weather condition (max temp lower than 80 but higher than 70, wind speed less than 10, 0 cloudiness)
narrowed_cities_df = weather_data_rename.loc[(weather_data_rename["Max Temp"] >= 70)& (weather_data_rename["Max Temp"] <= 80) & (weather_data_rename["Wind Speed"]<= 10) &(weather_data["Cloudiness"] ==0)].dropna()
narrowed_cities_df

City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
187              moroni -11.70   43.26     76.95        73           0   
189                duku  11.24    4.91     78.39        59           0   
213         chake chake  -5.25   39.77     74.43        83           0   
275  humberto de campos  -2.60  -43.46     79.00        78           0   
324             fayaoue -20.65  166.53     76.53        72           0   
337              moerai -22.43 -151.33     77.68        80           0   
360         grand gaube -20.01   57.66     73.99        71           0   
488                qena  26.16   32.72     77.00        23           0   
528                 hit  33.64   42.83     72.82        35           0   

     Wind Speed Country        Date  
187        2.53      KM  1603241189  
189        3.71      NG  1603241192  
213        6.64      TZ  1603241222  
275        8.81      BR  1603241301  
324        6.96      NC  1603241364  
337        6.46      PF  1603241379  
360        7.00      MU  1603241409  
488        4.70      EG  1603241578  
528        7.92      IQ  1603241383

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
# Store variable named hotel_df
hotel_df = narrowed_cities_df.loc[:, ["City", "Country", "Lat", "Lng"]]

# Add "Hotel Name" to DataFrame
hotel_df["Hotel Name"] = ""

# Search for Hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
          "key": g_key}

In [73]:
# Creat the loop to find our hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lng},{lat}"
    print({city_name})
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        time.sleep(1)
        
    except (KeyError, IndexError):
        print("Missing result")

{'moroni'}
Missing result
{'duku'}
Missing result
{'chake chake'}
Closest hotel in chake chake is La Almazara de Valdeverdeja.
{'humberto de campos'}
Missing result
{'fayaoue'}
Missing result
{'moerai'}
Missing result
{'grand gaube'}
Missing result
{'qena'}
Missing result
{'hit'}
Missing result


In [74]:
hotel_df

City Country    Lat     Lng                   Hotel Name
187              moroni      KM -11.70   43.26                             
189                duku      NG  11.24    4.91                             
213         chake chake      TZ  -5.25   39.77  La Almazara de Valdeverdeja
275  humberto de campos      BR  -2.60  -43.46                             
324             fayaoue      NC -20.65  166.53                             
337              moerai      PF -22.43 -151.33                             
360         grand gaube      MU -20.01   57.66                             
488                qena      EG  26.16   32.72                             
528                 hit      IQ  33.64   42.83

In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [76]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))